In [24]:
##########-DATASET-##########
from google.colab import files
uploaded = files.upload() 

Saving rus.txt to rus (1).txt


In [51]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

def deaccent(value):
  from unicodedata import normalize, combining
  value = normalize('NFD', value)
  value = u''.join(c for c in value if not combining(c))
  return value

def load_document(name):
	file = open(name, mode='rt', encoding='utf-8')
	text = file.read()
	file.close()
	return text

def sep_to_pairs(document):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

def clean_all_pairs(lines):
	cleaned = list()
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# normalize unicode characters
			line = deaccent(line)#normalize('NFC', line).encode('ascii', 'ignore') 
			#line = line.decode('UTF-8')
			# tokenize on white space
			line = line.split()
			# convert to lowercase
			line = [word.lower() for word in line]
			# remove punctuation from each token
			line = [word.translate(table) for word in line]
			# remove non-printable chars form each token
			#line = [re_print.sub('', w) for w in line]
			# remove tokens with numbers in them
			line = [word for word in line if word.isalpha()]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

def save_data(sentences, name):
	dump(sentences, open(name, 'wb'))
	print('Saved: %s' % name)

name = 'rus.txt'
doc = load_document(name)
pairs = sep_to_pairs(doc)
clean_pairs = clean_all_pairs(pairs)
save_data(clean_pairs, 'english-russian.pkl')
for i in range(100):
	print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

Saved: english-russian.pkl
[go] => [иди]
[go] => [идите]
[hi] => [здравствуите]
[hi] => [привет]
[hi] => [хаи]
[hi] => [здрасте]
[hi] => [здорово]
[run] => [беги]
[run] => [бегите]
[run] => [беги]
[run] => [бегите]
[who] => [кто]
[fire] => [огонь]
[fire] => [пожар]
[help] => [помогите]
[help] => [на помощь]
[help] => [спасите]
[jump] => [прыгаи]
[jump] => [прыгаите]
[jump] => [прыгаи]
[stop] => [стои]
[stop] => [остановитесь]
[stop] => [остановись]
[wait] => [подожди]
[wait] => [подождите]
[wait] => [ждите]
[wait] => [жди]
[wait] => [ждите]
[do it] => [сделаи это]
[go on] => [продолжаи]
[go on] => [продолжаите]
[hello] => [здравствуите]
[hello] => [привет]
[hello] => [алло]
[hurry] => [поспешите]
[i ran] => [я бежал]
[i ran] => [я бежала]
[i ran] => [я побежал]
[i ran] => [я побежала]
[i see] => [понимаю]
[i see] => [понятно]
[i see] => [вижу]
[i try] => [я пытаюсь]
[i try] => [я стараюсь]
[i try] => [я пробую]
[i won] => [я победил]
[i won] => [я победила]
[i won] => [я выиграл]
[i wo

In [52]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-russian.pkl')

# reduce dataset size
n_sentences = 10000
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[:9000], dataset[9000:]
# save
save_clean_data(dataset, 'english-russian-both.pkl')
save_clean_data(train, 'english-russian-train.pkl')
save_clean_data(test, 'english-russian-test.pkl')

Saved: english-russian-both.pkl
Saved: english-russian-train.pkl
Saved: english-russian-test.pkl


In [53]:
##########-MODEL-##########
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot

from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

def create_new_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

def max_length_in_list(lines):
	return max(len(line.split()) for line in lines)

def encode_sequences(tokenizer, length, lines):
	# encode sequences to integer 
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

dataset = load_clean_sentences('english-russian-both.pkl')
trainset = load_clean_sentences('english-russian-train.pkl')
testset = load_clean_sentences('english-russian-test.pkl')

eng_tokenizer = create_new_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length_in_list(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
rus_tokenizer = create_new_tokenizer(dataset[:, 1])
rus_vocab_size = len(rus_tokenizer.word_index) + 1
rus_length = max_length_in_list(dataset[:, 1])
print('Russian Vocabulary Size: %d' % rus_vocab_size)
print('Russian Max Length: %d' % (rus_length))

trainX = encode_sequences(rus_tokenizer, rus_length, trainset[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, trainset[:, 0])
trainY = encode_output(trainY, eng_vocab_size)

testX = encode_sequences(rus_tokenizer, rus_length, testset[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, testset[:, 0])
testY = encode_output(testY, eng_vocab_size)

model = define_model(rus_vocab_size, eng_vocab_size, rus_length, eng_length, 256)
model.compile(optimizer='ADAM', loss='categorical_crossentropy')
print(model.summary())
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=35, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=20)
outputs = [layer.output for layer in model.layers]  
print(outputs)

English Vocabulary Size: 1838
English Max Length: 5
Russian Vocabulary Size: 4733
Russian Max Length: 8
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 8, 256)            1211648   
_________________________________________________________________
lstm_7 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 1838)           472366    
Total params: 2,734,638
Trainable params: 2,734,638
Non-trainable params: 0
_________________________________________________________________
None
Train on 9000 samples, 


Epoch 00035: val_loss did not improve from 1.65420
[<tf.Tensor 'embedding_4/GatherV2:0' shape=(?, 8, 256) dtype=float32>, <tf.Tensor 'lstm_7/TensorArrayReadV3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'repeat_vector_4/Tile:0' shape=(?, 5, 256) dtype=float32>, <tf.Tensor 'lstm_8/transpose_1:0' shape=(?, ?, 256) dtype=float32>, <tf.Tensor 'time_distributed_4/Reshape_1:0' shape=(?, 5, 1838) dtype=float32>]


In [54]:
##########-EVALUATION-##########
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction


chencherry = SmoothingFunction()
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

def max_length(lines):
	return max(len(line.split()) for line in lines)

def encode_sequences(tokenizer, length, lines):
	X = tokenizer.texts_to_sequences(lines)
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, eng_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append(raw_target.split())
		predicted.append(translation.split())
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

dataset = load_clean_sentences('english-russian-both.pkl')
trainset = load_clean_sentences('english-russian-train.pkl')
testset = load_clean_sentences('english-russian-test.pkl')
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
rus_tokenizer = create_tokenizer(dataset[:, 1])
rus_vocab_size = len(rus_tokenizer.word_index) + 1
rus_length = max_length(dataset[:, 1])
trainX = encode_sequences(rus_tokenizer, rus_length, trainset[:, 1])
testX = encode_sequences(rus_tokenizer, rus_length, testset[:, 1])
model = load_model('model.h5')
print('train')
evaluate_model(model, eng_tokenizer, trainX, trainset)
print('test')
evaluate_model(model, eng_tokenizer, testX, testset)

train
src=[он не умеет петь], target=[he cant sing], predicted=[he cant sing]
src=[перестаньте снимать], target=[stop filming], predicted=[stop filming]
src=[том болеет], target=[tom is sick], predicted=[tom is]
src=[вы устали], target=[youre tired], predicted=[youre tired]
src=[не открываите ее], target=[dont open it], predicted=[dont open it]
src=[я твои], target=[im yours], predicted=[im yours]
src=[сделаите еще раз], target=[do that again], predicted=[do that again]
src=[ешьте не спеша], target=[eat slowly], predicted=[eat slowly]
src=[будь объективен], target=[be objective], predicted=[be objective]
src=[мы с тобои ровесники], target=[im your age], predicted=[im your age]


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU-1: 0.090809
BLEU-2: 0.286736
BLEU-3: 0.454176
BLEU-4: 0.509518
test
src=[увидимся], target=[see you], predicted=[are you then]
src=[жизнь забавна], target=[life is fun], predicted=[mary is hard]
src=[они обожглись], target=[they burned], predicted=[theyre boys]
src=[теперь уходи], target=[now go away], predicted=[now get out]
src=[идите без меня], target=[go without me], predicted=[come without me]
src=[прекрати пожалуиста], target=[please stop], predicted=[please stop]
src=[это порок], target=[its a vice], predicted=[its is]
src=[просто будь счастлив], target=[just be happy], predicted=[just be happy]
src=[я приготовила ужин], target=[i made supper], predicted=[i made dinner]
src=[у меня диабет], target=[im diabetic], predicted=[i am diabetic]
BLEU-1: 0.081189
BLEU-2: 0.270323
BLEU-3: 0.437357
BLEU-4: 0.493259
